In [1]:
from urllib.request import urlretrieve
from os.path import isfile, isdir
import problem_unittests as tests
import tarfile
from sklearn.preprocessing import LabelBinarizer
encoder = LabelBinarizer()
encoder.fit(range(10))
# Use Floyd's cifar-10 dataset if present
cifar10_dataset_folder_path = 'cifar-10-batches-py'

floyd_cifar10_location = '/input/cifar-10/python.tar.gz'
if isfile(floyd_cifar10_location):
    tar_gz_path = floyd_cifar10_location
else:
    tar_gz_path = 'cifar-10-python.tar.gz'

import helper
import numpy as np

# Explore the dataset
batch_id = 1
sample_id = 5
helper.display_stats(cifar10_dataset_folder_path, batch_id, sample_id)
#normalize the data 
def normalize(x):
    return x/255
#one hot encoding
def one_hot_encode(x):
    #try using np.eye as well 
    return encoder.transform(x)


# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(cifar10_dataset_folder_path, normalize, one_hot_encode)


import pickle
import helper

# Load the Preprocessed Validation data
valid_features, valid_labels = pickle.load(open('preprocess_validation.p', mode='rb'))

#build the model
import tensorflow as tf

def neural_net_image_input(image_shape):
    imw, imh, cc = image_shape
    x = tf.placeholder(tf.float32,
    shape=[None, imw, imh, cc], name='x')
    return x

def neural_net_label_input(n_classes):
    y = tf.placeholder(tf.float32,
    shape=[None, n_classes], name='y')
    return y


def neural_net_keep_prob_input():
    k = tf.placeholder(tf.float32, name='keep_prob')
    return k


tf.reset_default_graph()
#convolution and maxpool layers
def conv2d_maxpool(x_tensor, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides):
    filter_size_width, filter_size_height = conv_ksize
    color_channels = x_tensor.get_shape().as_list()[-1]
    weight = tf.Variable(tf.truncated_normal(
        [filter_size_width, filter_size_height, color_channels, conv_num_outputs], stddev=0.01))
    bias = tf.Variable(tf.zeros(conv_num_outputs))
   
    # Apply Convolution
    # set the stride for batch and input_channels (i.e. the first and fourth element in the strides array) to be 1.
    a, b = conv_strides
    full_conv_strides = [1, a, b, 1]
    conv_layer = tf.nn.conv2d(x_tensor, weight, strides=full_conv_strides, padding='SAME')
    # Add bias
    conv_layer = tf.nn.bias_add(conv_layer, bias)
    # Apply activation function
    conv_layer = tf.nn.relu(conv_layer)
    # Apply Max Pooling
    a, b = pool_strides
    full_pool_strides = [1, a, b, 1]
    a, b = pool_ksize
    full_pool_ksize = [1, a, b, 1]
    conv_layer = tf.nn.max_pool(conv_layer,
        ksize=full_pool_ksize,strides=full_pool_strides,
        padding='SAME')
    return conv_layer 
#flatten layer

def flatten(x_tensor):
    shape = x_tensor.get_shape().as_list()
    ns = np.prod(shape[1:])
    flat = tf.reshape(x_tensor, [-1, ns])
    return flat
#fully connected layer
def fully_conn(x_tensor, num_outputs):
    weight = tf.Variable(tf.truncated_normal(
        [x_tensor.get_shape().as_list()[-1] ,num_outputs], stddev=0.01))
    bias = tf.Variable(tf.zeros(num_outputs))
    fc = tf.add(tf.matmul(x_tensor, weight), bias)
    fc = tf.nn.relu(fc)
    return fc

def output(x_tensor, num_outputs):
    weight = tf.Variable(tf.truncated_normal(
        [x_tensor.get_shape().as_list()[-1] , num_outputs], stddev=0.01))
    bias = tf.Variable(tf.zeros(num_outputs))
    out = tf.add(tf.matmul(x_tensor, weight), bias)
    return out

def conv_net(x, keep_prob):
    
    conv_num_outputs = 32
    conv_ksize = (5,5)
    conv_strides = (1,1) # tried (3,3) as well
    pool_ksize = (2,2)
    pool_strides = (2,2)
    logits = conv2d_maxpool(x, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides)
    
    conv_num_outputs = 64
    conv_ksize = (5,5)
    conv_strides = (1,1)
    pool_ksize = (2,2)
    pool_strides = (2,2)
    logits = conv2d_maxpool(logits, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides)
    
    conv_num_outputs = 128
    conv_ksize = (5,5)
    conv_strides = (1,1)
    pool_ksize = (2,2)
    pool_strides = (2,2)
    logits = conv2d_maxpool(logits, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides)

    conv_num_outputs = 256
    conv_ksize = (5,5)
    conv_strides = (1,1) # earlier 3,3 
    pool_ksize = (2,2)
    pool_strides = (2,2)
    logits = conv2d_maxpool(logits, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides)
    # gave just 46 % with this layer included at dropout = 0.69 w/0 a third fcc layer
    # increased  validation accuracy with droput at 0.6
    tf.nn.dropout(logits, keep_prob)    
    logits = flatten(logits)
    
    num_outputs = 512
    logits = fully_conn(logits, num_outputs)
    
    num_outputs = 512
    logits = fully_conn(logits, num_outputs)
    
    num_outputs = 512
    logits = fully_conn(logits, num_outputs)
    # not included initialy    
    
    num_outputs = 10
    logits = output(logits, num_outputs)
    
    return logits


# Remove previous weights, bias, inputs, etc..
tf.reset_default_graph()

# Inputs
x = neural_net_image_input((32, 32, 3))
y = neural_net_label_input(10)
keep_prob = neural_net_keep_prob_input()

# Model
logits = conv_net(x, keep_prob)

# Name logits Tensor, so that is can be loaded from disk after training
logits = tf.identity(logits, name='logits')

# Loss and Optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.AdamOptimizer().minimize(cost)

# Accuracy
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32), name='accuracy')


def train_neural_network(session, optimizer, keep_probability, feature_batch, label_batch):
    session.run(optimizer, feed_dict={
                x: feature_batch,
                y: label_batch,
                keep_prob: keep_probability})
    


def print_stats(session, feature_batch, label_batch, cost, accuracy):
    validation_loss = sess.run(cost, feed_dict={
        x: valid_features,
        y: valid_labels,
        keep_prob: 1})
    validation_accuracy = sess.run(accuracy, feed_dict={
        x: valid_features,
        y: valid_labels,
        keep_prob: 1})
    print("validation_loss: {}, validation_accuracy: {}".format(validation_loss, validation_accuracy))

# Tune Parameters
epochs = 25 #trial and error
batch_size = 128
keep_probability = 0.50 #changed from 0.69 works great !

# epochs = 35 #trial and error
# batch_size = 256
# keep_probability = 0.50 #change


print('Checking the Training on a Single Batch...')
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    # Training cycle
    for epoch in range(epochs):
        batch_i = 1
        for batch_features, batch_labels in helper.load_preprocess_training_batch(batch_i, batch_size):
            train_neural_network(sess, optimizer, keep_probability, batch_features, batch_labels)
        print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, batch_i), end='')
        print_stats(sess, batch_features, batch_labels, cost, accuracy)


save_model_path = './image_classification'

print('Training...')
with tf.Session() as sess:
    # Initializing the variables
    sess.run(tf.global_variables_initializer())
    
    # Training cycle
    for epoch in range(epochs):
        # Loop over all batches
        n_batches = 5
        for batch_i in range(1, n_batches + 1):
            for batch_features, batch_labels in helper.load_preprocess_training_batch(batch_i, batch_size):
                train_neural_network(sess, optimizer, keep_probability, batch_features, batch_labels)
            print('Epoch {:>2}, CIFAR-10 Batch {}:  '.format(epoch + 1, batch_i), end='')
            print_stats(sess, batch_features, batch_labels, cost, accuracy)
            
    # Save Model
    saver = tf.train.Saver()
    save_path = saver.save(sess, save_model_path)

#     try one to one convolution just for a try 
#     conv_num_outputs = 128
#     conv_ksize = (1,1)
#     conv_strides = (1,1)
#     pool_ksize = (2,2)
#     pool_strides = (2,2)
#     logits = conv2d_maxpool(logits, conv_num_outputs, conv_ksize, conv_strides, pool_ksize, pool_strides)
#horribly bad    


Stats of batch 1:
Samples: 10000
Label Counts: {0: 1005, 1: 974, 2: 1032, 3: 1016, 4: 999, 5: 937, 6: 1030, 7: 1001, 8: 1025, 9: 981}
First 20 Labels: [6, 9, 9, 4, 1, 1, 2, 7, 8, 3, 4, 7, 7, 2, 9, 9, 9, 3, 2, 6]

Example of Image 5:
Image - Min Value: 0 Max Value: 252
Image - Shape: (32, 32, 3)
Label - Label Id: 1 Name: automobile
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

Checking the Training on a Single Batch...
Epoch  1, CIFAR-10 Batch 1:  validation_loss: 2.302511215209961, validation_accuracy: 0.10379999876022339
Epoch  2, CIFAR-10 Batch 1:  validation_loss: 2.1308491230010986, validation_accuracy: 0.17139999568462372
Epoch  3, CIFAR-10 Batch 1:  val

In [1]:
#test

import tensorflow as tf
import pickle
import helper
import random

# Set batch size if not already set
try:
    if batch_size:
        pass
except NameError:
    batch_size = 64

save_model_path = './image_classification'
n_samples = 4
top_n_predictions = 3

def test_model():
    """
    Test the saved model against the test dataset
    """

    test_features, test_labels = pickle.load(open('preprocess_test.p', mode='rb'))
    loaded_graph = tf.Graph()

    with tf.Session(graph=loaded_graph) as sess:
        # Load model
        loader = tf.train.import_meta_graph(save_model_path + '.meta')
        loader.restore(sess, save_model_path)

        # Get Tensors from loaded model
        loaded_x = loaded_graph.get_tensor_by_name('x:0')
        loaded_y = loaded_graph.get_tensor_by_name('y:0')
        loaded_keep_prob = loaded_graph.get_tensor_by_name('keep_prob:0')
        loaded_logits = loaded_graph.get_tensor_by_name('logits:0')
        loaded_acc = loaded_graph.get_tensor_by_name('accuracy:0')
        
        # Get accuracy in batches for memory limitations
        test_batch_acc_total = 0
        test_batch_count = 0
        
        for test_feature_batch, test_label_batch in helper.batch_features_labels(test_features, test_labels, batch_size):
            test_batch_acc_total += sess.run(
                loaded_acc,
                feed_dict={loaded_x: test_feature_batch, loaded_y: test_label_batch, loaded_keep_prob: 1.0})
            test_batch_count += 1

        print('Testing Accuracy: {}\n'.format(test_batch_acc_total/test_batch_count))

        # Print Random Samples
        random_test_features, random_test_labels = tuple(zip(*random.sample(list(zip(test_features, test_labels)), n_samples)))
        random_test_predictions = sess.run(
            tf.nn.top_k(tf.nn.softmax(loaded_logits), top_n_predictions),
            feed_dict={loaded_x: random_test_features, loaded_y: random_test_labels, loaded_keep_prob: 1.0})
        helper.display_image_predictions(random_test_features, random_test_labels, random_test_predictions)


test_model()

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./image_classification
Testing Accuracy: 0.5892714968152867



In [2]:
########
def display_stats(cifar10_dataset_folder_path, batch_id, sample_id):
"""
#Display Stats of the the dataset
"""
batch_ids = list(range(1, 6))
if batch_id not in batch_ids:
print('Batch Id out of Range. Possible Batch Ids: {}'.format(batch_ids))
return None
features, labels = load_cfar10_batch(cifar10_dataset_folder_path, batch_id)
if not (0 <= sample_id < len(features)):
print('{} samples in batch {}. {} is out of range.'.format(len(features)
return None
print('\nStats of batch {}:'.format(batch_id))
print('Samples: {}'.format(len(features)))
print('Label Counts: {}'.format(dict(zip(*np.unique(labels, return_counts=True
print('First 20 Labels: {}'.format(labels[:20]))
sample_image = features[sample_id]
sample_label = labels[sample_id]
label_names = _load_label_names()
print('\nExample of Image {}:'.format(sample_id))
print('Image - Min Value: {} Max Value: {}'.format(sample_image.min(), sample_
print('Image - Shape: {}'.format(sample_image.shape))
print('Label - Label Id: {} Name: {}'.format(sample_label, label_names[sample_
plt.axis('off')
                                                                       
                                                                       plt.imshow(sample_image)

def _preprocess_and_save(normalize, one_hot_encode, features, labels, filename):

#Preprocess data and save it to file

features = normalize(features)
labels = one_hot_encode(labels)
pickle.dump((features, labels), open(filename, 'wb'))
def preprocess_and_save_data(cifar10_dataset_folder_path, normalize, one_hot_encode):

#Preprocess Training and Validation Data

n_batches = 5
valid_features = []
valid_labels = []
for batch_i in range(1, n_batches + 1):
features, labels = load_cfar10_batch(cifar10_dataset_folder_path, batch_i
validation_count = int(len(features) * 0.1)
# Prprocess and save a batch of training data
def _preprocess_and_save(normalize,one_hot_encode,
features[:-validation_count],
labels[:-validation_count],
'preprocess_batch_' + str(batch_i) + '.p')
# Use a portion of training batch for validation
valid_features.extend(features[-validation_count:])
valid_labels.extend(labels[-validation_count:])
# Preprocess and Save all validation data
_preprocess_and_save(
normalize,
one_hot_encode,
np.array(valid_features),
np.array(valid_labels),
'preprocess_validation.p')
with open(cifar10_dataset_folder_path + '/test_batch', mode='rb') as file:
batch = pickle.load(file, encoding='latin1')
# load the training data
test_features = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpo
test_labels = batch['labels']
# Preprocess and Save all training data
def _preprocess_and_save(
normalize,
one_hot_encode,
np.array(test_features),

Image_classification

_preprocess_and_save(normalize, one_hot_encode, features, labels, filename):


np.array(test_labels),
'preprocess_training.p')
def batch_features_labels(features, labels, batch_size):
"""
#Split features and labels into batches
"""
for start in range(0, len(features), batch_size):
end = min(start + batch_size, len(features))
yield features[start:end], labels[start:end]
def load_preprocess_training_batch(batch_id, batch_size):
"""
#Load the Preprocessed Training data and return them in batches of <batch_size
"""
filename = 'preprocess_batch_' + str(batch_id) + '.p'
features, labels = pickle.load(open(filename, mode='rb'))
# Return the training data in batches of size <batch_size> or less
return batch_features_labels(features, labels, batch_size)
def display_image_predictions(features, labels, predictions):
n_classes = 10
label_names = _load_label_names()
label_binarizer = LabelBinarizer()
label_binarizer.fit(range(n_classes))
label_ids = label_binarizer.inverse_transform(np.array(labels))
fig, axies = plt.subplots(nrows=4, ncols=2)
fig.tight_layout()
fig.suptitle('Softmax Predictions', fontsize=20, y=1.1)
n_predictions = 3
margin = 0.05
ind = np.arange(n_predictions)
width = (1. - 2. * margin) / n_predictions
for image_i, (feature, label_id, pred_indicies, pred_values) in enumerate(zip
pred_names = [label_names[pred_i] for pred_i in pred_indicies]
correct_name = label_names[label_id]
axies[image_i][0].imshow(feature*255)
axies[image_i][0].set_title(correct_name)
axies[image_i][0].set_axis_off()
axies[image_i][1].barh(ind + margin, pred_values[::-1], width)
axies[image_i][1].set_yticks(ind + margin)
axies[image_i][1].set_yticklabels(pred_names[::-1])
axies[image_i][1].set_xticks([0, 0.5, 1.0])


IndentationError: expected an indented block (<ipython-input-2-ef24d0d1c60c>, line 5)